# Topic Modeling mit Gensim

Dieses Skript erstellt ein Topic Model mit Gensim. 

* **gensim** ist eine Bibliothek, die das Topic Model berechnet
* **nltk** oder **spacy** um Text vorzuverarbeiten
* **os** ist eine Bibliothek, um das Korpus zu laden


## Vorbereitung

In [ ]:
# Import von Bibliotheken, die wir verwenden

import gensim, nltk, os, spacy
from pathlib import Path

In [ ]:
# Liste der Files ausgeben, die analysiert werden

files = sorted(os.listdir('fedpapers'))
files

In [ ]:
# Erstellung von Variablen, die unser Input-Korpus enthalten
texts = [] # Liste der Texte
labels = [] # Labels mit den Dateititeln

In [ ]:
# Spacy Modell muss installiert sein, z.B. en_core_web_md, siehe spacy-Modelle für Englisch: https://spacy.io/models/en
# Installation über die Powershell "python -m spacy download en_core_web_md"

# Spacy-Modell und Stopwörter laden

nlp = spacy.load("en_core_web_md")
#nlp = spacy.load('en_core_web_sm')

stopwords = nlp.Defaults.stop_words

print(len(stopwords))
print(stopwords)

#https://machinelearningknowledge.ai/tutorial-for-stopwords-in-spacy/


## Vorverarbeitung der Texte

In [ ]:
# Verwendung der os-Bibliothek, um die Files zu laden. 
# Iteriert über alle Dokumente im Korpus-Folder.
for root, dirs, files in os.walk('fedpapers'): 
    for file_name in files: # iteriert über jede Datei, lädt und öffnet sie in runtime 
        with open(os.path.join(root, file_name), encoding='utf-8') as rf: #plattformunabhängiger Weg, um die Files zu öffnen
            text = nlp(rf.read()) #String aus der Datei bekommen, weil wir den Text in individuelle Tokens teilen wollen.
            cleaned = [token.text for token in text if not token.is_punct ] #Textbereinigung mit spacy
            texts.append(cleaned)
            labels.append(file_name[:-4]) #speichert die Filenamen (ohne Dateinamen .txt)

# Ersten Text ausgeben    
print(texts[:1])

# Alternativ dazu kann zur Textbereinigung auch NLTK verwendet werden
#            tokens = nltk.word_tokenize(text)
#            cleaned = [word for word in tokens if word.isalnum()] 
#            texts.append(cleaned)
#            labels.append(file_name[:-4]) #speichert Information die es erlaubt zu labeln; letzten vier Zeichen werden weggeschnitten.
#print(texts[:1])

## Topic Modeling 

Verwendung von gensim, einer Python Bibliothek für Topic Modeling, Dokumentenindizierung, etc., um das Topic Modeling durchzuführen. Dabei wird das Korpus in ein gensim.corpora.Dictionary transformiert, um es anschließend mit Gensim verarbeiten zu können. 

Zunächst muss gensim über die Powershell installiert werden

In [ ]:
# !pip install gensim

In [ ]:
from gensim.models import LdaModel

In [ ]:
from gensim import corpora

# Erstellt ein Gensim dictionary aus den Texten
dictionary = corpora.Dictionary(texts)

# Konvertiert dictionary in ein bag-of-words Korpus
corpus = [dictionary.doc2bow(text) for text in texts]

# Erstellt LDA model unter Verwendung von corpus und dictionary
from gensim.models import LdaModel

num_topics = 10
chunksize = 2000
passes = 20
iterations = 400
eval_every = None 

# Index zum Wort dictionary.
temp = dictionary[0]  # Laden des dictionary.
id2word = dictionary.id2token

model = LdaModel(
    corpus=corpus, # das Korpus
    id2word=id2word,# mapping der id-Zahlen zu den Wörtern
    chunksize=chunksize, #Chunkgröße
    alpha='auto',
    eta='auto',
    iterations=iterations,#Anzahl der Iterationen
    num_topics=num_topics,Anzahl der Topics
    passes=passes,
    eval_every=eval_every
)



In [ ]:
model.save("topicmodel-lda.p")

In [ ]:
# Bei der Erstellung des Topic Models wurden im Folder Dateien mit dem Präfix fed_ gespeichert, z.B. das Korpus, die Topicverteilung in den Dokumenten, die Top-Keywords in den Topics etc. Diese Dateien können zur Weiterverarbeitung und Visualisierung verwendet werden.
# Die Informationen können aber auch direkt hier angezeigt werden, z.B. die Topics mit den Top 20 Wörtern
topics = model.show_topics(num_topics=num_topics, num_words=20)
print (topics)

In [ ]:
# Hier etwas übersichtlicher
for topic in topics:
    print(f"Topic #{topic[0]}: {topic[1]}...\n\n")
    

### Visualisierungsmöglichkeiten
Der letzte Teil des Topic Modeling Workflows besteht aus der Visualisierung der Ergebnisse. Es gibt unzählige Visulisierungen, die wiederverwendet und angepasst werden können, z.B. hier: https://www.youtube.com/watch?v=GLfOANUqulA&t=19s und https://www.youtube.com/watch?v=GLfOANUqulA&t=19s

In [ ]:
# Wordcloud erstellen

# Dazu muss zunächst über die Powershell wordcloud installiert werden mit pip install wordcloud

from matplotlib import pyplot as plt
from wordcloud import WordCloud, STOPWORDS
import matplotlib.colors as mcolors

cols = [color for name, color in mcolors.TABLEAU_COLORS.items()]  # more colors: 'mcolors.XKCD_COLORS'

cloud = WordCloud(background_color='white',
                  width=2500,
                  height=1800,
                  max_words=10,
                  colormap='tab10',
                  color_func=lambda *args, **kwargs: cols[i],
                  prefer_horizontal=1.0)

topics = model.show_topics(formatted=False)

fig, axes = plt.subplots(2, 3, figsize=(10,10), sharex=True, sharey=True) # Anzahl der Plots kann hier erhöht werden. 

for i, ax in enumerate(axes.flatten()):
    fig.add_subplot(ax)
    topic_words = dict(topics[i][1])
    cloud.generate_from_frequencies(topic_words, max_font_size=300)
    plt.gca().imshow(cloud)
    plt.gca().set_title('Topic ' + str(i), fontdict=dict(size=16))
    plt.gca().axis('off')


plt.subplots_adjust(wspace=0, hspace=0)
plt.axis('off')
plt.margins(x=0, y=0)
plt.tight_layout()
plt.show()
plt.savefig('wordcloud.png', dpi=300)